In [6]:
import os
import alpaca_trade_api as tradeapi
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import pandas_ta as ta
import tkinter as tk
from tkinter import messagebox

In [7]:
load_dotenv()

True

In [8]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
base_url = os.getenv("BASE_URL")

In [9]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url,
    api_version="v2"
)

In [10]:
# Define the list of stock tickers
tickers = ['PANW', 'LRCX', 'AAPL', 'AMZN', 'MSFT', 'TSLA', 'CAT', 'DE']

In [11]:
# Define Bollinger band parameters
period = 20
std_dev = 2

In [15]:
def get_bollinger_bands(ticker):
    # Get historical data for the ticker
    historical_data = api.get_bars(ticker, 'day', limit=period+1).df[ticker]

In [ ]:
# Extract closing prices
closing_prices = historical_data.close.values

In [ ]:
# Calculate Bollinger bands
bollinger_bands = ta.bbands(pd.Series(closing_prices), length=period, std=std_dev)

return bollinger_bands

In [ ]:
def submit_trade(ticker, action):
    try:
        # Check if the submitted ticker is valid
        if ticker in tickers:
            # Get the current price for the ticker
            current_price = api.get_last_trade(ticker).price

            # Check if the action is valid (buy or sell)
            if action == 'buy':
                # Place a buy order
                api.submit_order(
                    symbol=ticker,
                    qty=1,
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )
                messagebox.showinfo("Success", f"Buy order for {ticker} placed successfully.")
            elif action == 'sell':
                # Place a sell order
                api.submit_order(
                    symbol=ticker,
                    qty=1,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )
                messagebox.showinfo("Success", f"Sell order for {ticker} placed successfully.")
            else:
                messagebox.showerror("Error", "Invalid action. Please choose 'buy' or 'sell'.")
        else:
            messagebox.showerror("Error", "Invalid ticker.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

In [ ]:
def create_interface():
    root = tk.Tk()
    root.title("Bollinger Band Trading Bot")

In [ ]:
# Create a LabelFrame for ticker selection
    ticker_frame = tk.LabelFrame(root, text="Select Ticker")
    ticker_frame.pack(padx=10, pady=10)

    ticker_var = tk.StringVar()
    ticker_dropdown = tk.OptionMenu(ticker_frame, ticker_var, *tickers)
    ticker_dropdown.pack(padx=10, pady=10)

In [ ]:
# Create a LabelFrame for action selection
    action_frame = tk.LabelFrame(root, text="Select Action")
    action_frame.pack(padx=10, pady=10)

    action_var = tk.StringVar()
    action_dropdown = tk.OptionMenu(action_frame, action_var, "Buy", "Sell")
    action_dropdown.pack(padx=10, pady=10)

    # Create a button to submit the trade
    submit_button = tk.Button(root, text="Submit", command=lambda: submit_trade(ticker_var.get(), action_var.get()))
    submit_button.pack(padx=10, pady=10)

    root.mainloop()

In [ ]:
if __name__ == '__main__':
    create_interface()